In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from env import robot_goal
import random
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


In [3]:
env = robot_goal()

In [4]:
"""env.reset()

done = False
total_reward = 0
simulation_start = 0
while simulation_start != 1:
    simulation_start = env.sim.startSimulation()
while not done:
   action = random.randint(0,2)
   obs, rew, done = env.step(action)
   total_reward += rew
   print(f"{obs} -> {rew}")
simulation_end = 0        
while simulation_end != 1:
    simulation_end = env.sim.stopSimulation()     
print(f"Total reward: {total_reward}")"""



'env.reset()\n\ndone = False\ntotal_reward = 0\nsimulation_start = 0\nwhile simulation_start != 1:\n    simulation_start = env.sim.startSimulation()\nwhile not done:\n   action = random.randint(0,2)\n   obs, rew, done = env.step(action)\n   total_reward += rew\n   print(f"{obs} -> {rew}")\nsimulation_end = 0        \nwhile simulation_end != 1:\n    simulation_end = env.sim.stopSimulation()     \nprint(f"Total reward: {total_reward}")'

In [5]:
num_inputs = 10
num_actions = 3

model = keras.Sequential([
    keras.layers.Dense(128, activation="relu",input_shape=(num_inputs,)),
    keras.layers.Dense(num_actions, activation="softmax")
])

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.01))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1408      
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 1795 (7.01 KB)
Trainable params: 1795 (7.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
def run_episode(max_steps_per_episode = 100):
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    simulation_start = 0
    while simulation_start != 1:
        simulation_start = env.sim.startSimulation()
    for i in tqdm(range(max_steps_per_episode)):
        state = np.reshape(state,(1, state.shape[0]))
        action_probs = model.predict(state, verbose=0)
        action = np.argmax(action_probs)
        nstate, reward, done = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs)
        rewards.append(reward)
        state = nstate
    simulation_end = 0        
    while simulation_end != 1:
        simulation_end = env.sim.stopSimulation()     
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

In [8]:
s,a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

100%|██████████| 100/100 [01:26<00:00,  1.15it/s]

Total reward: 19.999999999999996


In [9]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

[array([[ 0.18060876, -0.04000501,  0.0268635 , ...,  0.15091164,
         -0.12753296,  0.00915667],
        [ 0.16119222,  0.1624736 , -0.1878345 , ...,  0.14091556,
         -0.04019155, -0.17210284],
        [ 0.0523188 ,  0.11685924,  0.06878991, ..., -0.05535288,
         -0.06048627,  0.05600075],
        ...,
        [ 0.02262938,  0.20555405,  0.15970235, ..., -0.02016139,
          0.13872905, -0.10778882],
        [ 0.15697779,  0.05459012, -0.02920412, ...,  0.1748731 ,
          0.14470752, -0.14837846],
        [ 0.11075254,  0.1204754 ,  0.03532745, ...,  0.00410755,
          0.1186709 ,  0.10154273]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [11]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(3)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    model.train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")
plt.plot(history)

100%|██████████| 100/100 [02:30<00:00,  1.50s/it]


0 -> 19.999999999999996


  3%|▎         | 3/100 [00:06<03:26,  2.13s/it]


KeyboardInterrupt: 